In [1]:
from antelope_catalog.providers import EcoinventLcia


In [2]:
L = EcoinventLcia('/data/LCI/LCIA implementation v3.1 2014_08_13.xlsx', ns_uuid='46802ca5-8b25-398c-af10-2376adaa4623')
%time L.load_all()
Lqi = L.make_interface('quantity')

local.ecoinvent.3.1.lcia: /data/LCI/LCIA implementation v3.1 2014_08_13.xlsx
Loading /data/LCI/LCIA implementation v3.1 2014_08_13.xlsx
702 new quantity entities added (702 total)
3255 new flow entities added (3255 total)
CPU times: user 24.7 s, sys: 254 ms, total: 25 s
Wall time: 25.1 s


In [5]:
recipes = [x for x in L.make_interface('index').lcia_methods(Method='recipe midpoint \(H\)$')]
mass = next(L.make_interface('index').quantities(Name='^mass'))

 [00] ReCiPe Midpoint (H), climate change, GWP100 [kg CO2-Eq] [LCIA]
 [01] ReCiPe Midpoint (H), terrestrial acidification, TAP100 [kg SO2-Eq] [LCIA]
 [02] ReCiPe Midpoint (H), photochemical oxidant formation, POFP [kg NMVOC] [LCIA]
 [03] ReCiPe Midpoint (H), particulate matter formation, PMFP [kg PM10-Eq] [LCIA]
 [04] ReCiPe Midpoint (H), fossil depletion, FDP [kg oil-Eq] [LCIA]
 [05] ReCiPe Midpoint (H), ozone depletion, ODPinf [kg CFC-11-Eq] [LCIA]
 [06] ReCiPe Midpoint (H), marine ecotoxicity, METPinf [kg 1,4-DCB-Eq] [LCIA]
 [07] ReCiPe Midpoint (H), natural land transformation, NLTP [m2] [LCIA]
 [08] ReCiPe Midpoint (H), water depletion, WDP [m3] [LCIA]
 [09] ReCiPe Midpoint (H), human toxicity, HTPinf [kg 1,4-DCB-Eq] [LCIA]
 [10] ReCiPe Midpoint (H), freshwater eutrophication, FEP [kg P-Eq] [LCIA]
 [11] ReCiPe Midpoint (H), metal depletion, MDP [kg Fe-Eq] [LCIA]
 [12] ReCiPe Midpoint (H), freshwater ecotoxicity, FETPinf [kg 1,4-DCB-Eq] [LCIA]
 [13] ReCiPe Midpoint (H), agricultura

In [6]:
from lcatools.qdb import Qdb

In [31]:
qdb = Qdb(quell_biogenic_CO2=True)
qdb._q.add_synonyms('Mass', mass.uuid)
for q in recipes:
    for cf in Lqi.factors(q):
        qdb.add_cf(cf)

Loading JSON data from /data/GitHub/lca-tools/lcatools/qdb/data/flowable_synlist.json:
Loading JSON data from /data/GitHub/lca-tools/lcatools/qdb/data/quantity_synlist.json:
local.qdb: /data/GitHub/lca-tools/lcatools/qdb/data/elcd_reference_quantities.json
Loading JSON data from /data/GitHub/lca-tools/lcatools/qdb/data/elcd_reference_quantities.json:
local.qdb: /data/GitHub/lca-tools/lcatools/providers/data/elcd_reference_quantities.json
24 new quantity entities added (24 total)
5 new flow entities added (5 total)


# Part II. Markets
For this next part I will look for markets that conceal variability.

In [12]:
from antelope_background.background import tarjan_background
from lcatools import archive_from_json, enum

In [16]:
%time e34c_i = archive_from_json('/data/LCI/Ecoinvent/3.4/cutoff_index.json.gz').make_interface('index')

Loading JSON data from /data/LCI/Ecoinvent/3.4/cutoff_index.json.gz:
local.ecoinvent.3.4.cutoff.index__20180904: /data/LCI/Ecoinvent/3.4/cutoff_index.json.gz
local.ecoinvent.3.4.cutoff: /data/LCI/Ecoinvent/3.4/ecoinvent 3.4_cutoff_ecoSpold02
19 new quantity entities added (19 total)
13459 new process entities added (13459 total)
4881 new flow entities added (4881 total)
CPU times: user 1.61 s, sys: 43.2 ms, total: 1.65 s
Wall time: 1.66 s


In [20]:
%time bg = tarjan_background(e34c_i, source='/data/LCI/Ecoinvent/3.4/cutoff_bg_tarjan.mat')

Loading JSON data from /data/LCI/Ecoinvent/3.4/cutoff_bg_tarjan.mat.index.json.gz:
CPU times: user 76.2 ms, sys: 9.25 ms, total: 85.5 ms
Wall time: 133 ms


In [27]:
from antelope_catalog.providers import EcospoldV2Archive
e34c_inv = EcospoldV2Archive('/data/LCI/Ecoinvent/3.4/ecoinvent 3.4_cutoff_ecoSpold02', prefix='datasets')

local.data.LCI.Ecoinvent.3.4.ecoinvent 3.4_cutoff_ecoSpold02: /data/LCI/Ecoinvent/3.4/ecoinvent 3.4_cutoff_ecoSpold02


In [109]:
q = recipes[0]

In [116]:
class MarketRangeAnalysis(object):
    """
    Used to store the results of an analysis of the range of LCIA results within a market process.
    """
    def __init__(self, market, methods):
        

In [2]:
a = [1, 2, 3]

In [7]:
xrange

NameError: name 'xrange' is not defined

In [108]:
def generate_nontrivial_markets():
    print('using %s to compute impacts' % q)
    for _market in e34c_i.processes(Name='^market for'):
        # currently no way to invert regex i.e. negative search
        if _market['Name'].startswith('market for electricity'):
            # these are obviously dealt with as proper mixes
            continue
        _flow_ref = _market.reference().flow.external_ref
        # this will be so much better when I'm using catalog refs
        _inv = [x for x in e34c_inv.get(_market.external_ref).inventory(_flow_ref) 
                if x.flow.external_ref == _flow_ref]
        if len(_inv) < 2:
            print('##  %2d ## %s' % (len(_inv), _market))
            #continue
        _vals = [t for t in filter(lambda x: x != 0, 
                                   (qdb.do_lcia(q, 
                                    bg.lci(x.termination, _flow_ref)).total() for x in _inv))]
        print(_vals)
        if len(set(_vals)) > 1:
            _ratio = max(abs(x) for x in _vals) / min(abs(x) for x in _vals)
            if 1 < _ratio < 2:
                continue
            _market.show()
            print('\nRatio: %10g | %s' % (_ratio, _vals))
            yield _market
        elif len(_vals) > 3:
            print('\n!!No Dynamic Range')
            for x in _inv:
                print(x.termination)
            yield _market

In [107]:
q.show()

Quantity Entity (ref ReCiPe Midpoint (H), marine eutrophication, MEP)
origin: local.ecoinvent.3.1.lcia
reference: [kg N-Eq]
     Name: ReCiPe Midpoint (H), marine eutrophication, MEP
  Comment: Ecoinvent LCIA implementation
 Category: marine eutrophication
Indicator: MEP
   Method: ReCiPe Midpoint (H)


In [117]:
_mg = generate_nontrivial_markets()

In [113]:
mkg = next(m for m in e34c_i.processes(Name='^market for'))

In [114]:
mkg.show()

Process Entity (ref 00215f8d-e206-416e-8f21-6c525bff21de)
origin: local.ecoinvent.3.4.cutoff
reference: market for blister-copper conversion facility [GLO] has Output:     blister-copper conversion facility [CPC: 53269: Other constructions for manufacturing] [unit]
            Name: market for blister-copper conversion facility
         Comment: no comment.
 Classifications: ['EcoSpold01Categories: metals/extraction', 'ISIC rev.4 ecoinvent: 4100a: Construction of factory buildings for the metal industry']
    SpatialScope: GLO
ParentActivityId: 4e4af77c-5a8e-4cae-980e-7c8d8ccb33b1
   TemporalScope: {'begin': '2011-01-01', 'end': '2017-12-31'}


In [111]:
mkt = next(_mg)

using ReCiPe Midpoint (H), climate change, GWP100 [kg CO2-Eq] [LCIA] to compute impacts
completed 65 iterations
Multiple CFs found: [25.0, 22.25]
Flow: Methane, non-fossil (CAS 000074-82-8) [urban air close to ground] [kg]
Quantity: ReCiPe Midpoint (H), climate change, GWP100 [kg CO2-Eq] [LCIA]
Multiple CFs found: [25.0, 22.25]
Flow: Methane, non-fossil (CAS 000074-82-8) [non-urban air or from high stacks] [kg]
Quantity: ReCiPe Midpoint (H), climate change, GWP100 [kg CO2-Eq] [LCIA]
Multiple CFs found: [22.25, 25.0]
Flow: Methane, non-fossil (CAS 000074-82-8) [unspecified] [kg]
Quantity: ReCiPe Midpoint (H), climate change, GWP100 [kg CO2-Eq] [LCIA]
completed 65 iterations
[7288277.5470186248, 7277001.2588517722]
completed 60 iterations
completed 61 iterations
[0.57776118919107422, 0.92762984465136278]
##   1 ## market for waste vapour barrier, flame-retarded [RoW]
completed 58 iterations
[-2.8507061626302304]
completed 67 iterations
completed 67 iterations
[6466.6517053064508, 6466.72

In [106]:
mkt = next(_mg)

completed 65 iterations
completed 65 iterations
[6385.5825032677694, 6369.8557680088434]
completed 60 iterations
completed 61 iterations
[0.00050770138395959182, 0.00098689853964844805]
##   1 ## market for waste vapour barrier, flame-retarded [RoW]
completed 58 iterations
[-0.0002036344547906391]
completed 67 iterations
completed 67 iterations
[6.1403574100075673, 6.1408773093283333]
completed 65 iterations
completed 65 iterations
[-0.00020509547612972847, -0.00020505582205357549]
completed 63 iterations
completed 63 iterations
[0.0093443936444337732, 0.0093425163768968752]
completed 68 iterations
completed 68 iterations
[645.65371101808398, 645.65371101808398]
completed 66 iterations
completed 66 iterations
[52.627047733388387, 54.274726912974387]
completed 64 iterations
exact result
completed 0 iterations
exact result
completed 0 iterations
[-0.0004621905202064463]
completed 64 iterations
completed 64 iterations
[0.0050691623725089329, 0.005072377040150379]
##   1 ## market for muni

In [115]:
next(_mg)

completed 67 iterations
completed 67 iterations
[0.014061531508380463, 0.020902335777321611]
exact result
completed 0 iterations
exact result
completed 0 iterations
[-28.747731397459201]
completed 68 iterations
completed 68 iterations
[91.601532172055002, 91.601532172055002]
##   1 ## market for barley grain, feed [GLO]
completed 69 iterations
[0.68111568139404732]
completed 66 iterations
completed 66 iterations
completed 66 iterations
completed 66 iterations
completed 66 iterations
completed 66 iterations
completed 66 iterations
completed 66 iterations
[0.42183928398462095, 0.47082241000353064, 0.41015717600535395, 0.4487512799504908, 0.40486876934441995, 0.43325426679920631, 0.37623629177793583, 0.39398454191757243]
completed 63 iterations
completed 64 iterations
[-1.3143215945506528, -1.5107557663518709]
completed 65 iterations
completed 65 iterations
[1155956.0011082985, 1224775.6068609839]
completed 68 iterations
completed 68 iterations
[53892474.389620282, 53892474.389620282]
## 

In [118]:
nontrivial_markets = [k for k in _mg]

using ReCiPe Midpoint (H), climate change, GWP100 [kg CO2-Eq] [LCIA] to compute impacts
completed 65 iterations
completed 65 iterations
[7288277.5470186248, 7277001.2588517722]
completed 60 iterations
completed 61 iterations
[0.57776118919107422, 0.92762984465136278]
##   1 ## market for waste vapour barrier, flame-retarded [RoW]
completed 58 iterations
[-2.8507061626302304]
completed 67 iterations
completed 67 iterations
[6466.6517053064508, 6466.7232655267117]
completed 65 iterations
completed 65 iterations
[-0.2187196590698833, -0.21867867717169445]
completed 63 iterations
completed 63 iterations
[6.156710844320342, 6.1560438901803876]
completed 68 iterations
completed 68 iterations
[438200.81361588161, 438200.81361588161]
completed 66 iterations
completed 66 iterations
[48260.855923469717, 49615.612355973055]
completed 64 iterations
exact result
completed 0 iterations
exact result
completed 0 iterations
[-2.0279985843016815]
completed 64 iterations
completed 64 iterations
[3.049401

KeyboardInterrupt: 

In [86]:
mkt.show_inventory()

market for electricity, low voltage [IN-Southern grid]
 0 Output: {*} [ ---  [kWh]]     electricity, low voltage [CPC: 17100: Electrical energy]


In [89]:
e34c_i._archive.ref

'local.ecoinvent.3.4.cutoff.index__20180904'

In [58]:
mkt.show()

Process Entity (ref 002d82f9-53b5-4f43-bce7-1960db0d5517)
origin: local.ecoinvent.3.4.cutoff
reference: market for coal tar [GLO] has Output:     coal tar [CPC: 34530: Roasted iron pyrites] [kg]
            Name: market for coal tar
         Comment: In this market, expert judgement was used to develop product specific transport distance estimations.
 Classifications: ['EcoSpold01Categories: oil/heating systems', 'ISIC rev.4 ecoinvent: 3530a: Steam and air conditioning supply based on liquid fuels ']
    SpatialScope: GLO
ParentActivityId: ad388f76-59fb-4278-97f3-9d8a2b65ce4c
   TemporalScope: {'begin': '2012-01-01', 'end': '2017-12-31'}


In [91]:
_flow_ref = mkt.reference().flow.external_ref
_flow_ref

'd69294d7-8d64-4915-a896-9996a014c410'

In [92]:
inv = enum(x for x in e34c_inv.get(mkt.external_ref).inventory(_flow_ref))

 [00]  Input:     [9.09e-09 [kg]] (#) sulfur hexafluoride, liquid (CAS 002551-62-4) [CPC: 342: Basic inorganic chemicals n.e.c.]
 [01]  Input:     [8.74e-08 [km]] (#) distribution network, electricity, low voltage [CPC: 532: Civil engineering works]
 [02]  Input:     [0.0312 [kWh]] (#) electricity, low voltage [CPC: 17100: Electrical energy]
 [03]  Input:     [0.969 [kWh]] (#) electricity, low voltage [CPC: 17100: Electrical energy]
 [04]  Input:     [0.0493 [kWh]] (o) electricity, low voltage [CPC: 17100: Electrical energy]
 [05] Output:     [9.09e-09 [kg]]     Sulfur hexafluoride (CAS 002551-62-4) [unspecified]


In [65]:
[z.flow.external_ref == _flow_ref for z in inv]

[False, False, True, True]